In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

/Users/tongqijun/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
vectorizer = TfidfVectorizer(max_features=250)

In [3]:
df=pd.read_csv("../data/atcoder_problem_tag_dataset.csv")
df=df[df["problem_texts"].notna()].reset_index()

def remove_section_title(txt):
    txt = txt.replace("\nProblem Statement","")
    txt = txt.replace("\nConstraints","")    
    txt = txt.replace("\nInput","")    
    txt = txt.replace("\nOutput","")
    return txt

df["concatenate_texts"]=df["problem_texts"]+" "+df["constraints"]+" "+df["input_texts"]+" "+df["output_texts"]
df["concatenate_texts"] = df["concatenate_texts"].apply(remove_section_title)
le = LabelEncoder()
df["tag"] = le.fit_transform(df["tag"])

In [4]:
df_copy = df.copy()
df_train = df_copy.sample(frac=0.8, random_state=0)
df_eval = df_copy.drop(df_train.index).reset_index()
df_train = df_train.reset_index()

In [5]:
X_train,X_eval,y_train,y_eval = train_test_split(vectorizer.fit_transform(df["concatenate_texts"]),df["tag"],random_state=0,test_size=0.2)

In [6]:
classifier = RandomForestClassifier(n_estimators=1000, random_state=0,n_jobs=-1)
classifier.fit(X_train, y_train) 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [7]:
y_pred = classifier.predict(X_eval)
print(accuracy_score(y_eval, y_pred))